In [1]:
import time
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import dispatcher
from pythonosc import osc_server
from pythonosc import udp_client
from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder

/Users/williamprimett1/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Session:
    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    W_A = np.zeros(win_size)
    W_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    delta_t=0.1*fs
    j = 0
    
#OSC input config   
ip = 'localhost'
port = 8001
server = osc_server.ThreadingOSCUDPServer(
  (ip, port), dispatcher)
    
# OSC output config
local_ip = '192.168.3.105'
local_port = 12000
output_address = '/results'
client = udp_client.SimpleUDPClient(local_ip, local_port)

In [3]:
#    Send to Processing/MAX/Etc...
def send_osc_bundle(dev_id, *values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    # Test bundle outputs: 1, 2, 3, ...
    for val in values:
        print (val)
        msg.add_arg(val)
        bundle.add_content(msg.build())
        bundle = bundle.build()
    client.send(bundle)
    
def output_individual(data, labels):
    data = json.loads(data)
    printJSON(data)
    bundle = osc_bundle_builder.OscBundleBuilder(
        osc_bundle_builder.IMMEDIATELY)

    for label in labels:
        output_address += self.device_number + "/bitalino/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        msg.add_arg(arg_to_add)
        msg.build()
        client.send(msg)

In [4]:
global sigA
global sigB
global W_A
global W_B

window = Session()

def bitalino_input_handler(unused_addr, *values):
    dataframe = []
    # Store and normalise data
    # Store and normalise data
    data = values
#     print (data)
#                        bit1.RESP1 = (bit1.val("RESP1")-512)/1024
    sigA = data[-2]
    sigB = data[-1]
    if window.j < window.no_buffer :
        i = window.j % window.no_buffer
        window.W_A[i*window.buffer_W:(i+1)*window.buffer_W]=sigA
        window.W_B[i*window.buffer_W:(i+1)*window.buffer_W]=sigB
        window.j = window.j+1

    else :
        print("window_A: ", window.W_A)
        window.W_A = np.roll(window.W_A,-window.buffer_W)
        window.W_A[-window.buffer_W:] = sigA
        print("window_A rolled: ", window.W_A)

        window.W_B = np.roll(window.W_B,-window.buffer_W)
        window.W_B[-window.buffer_W:] = sigB
        results = []
        result_1=syncm.lin_reg_r_metric(window.W_A,window.W_B)
        result_2=syncm.cos_similarity(window.W_A,window.W_B)

        #add results here
        results = [result_1, result_2.flat[0]]
#         resuts {
#             'reg': syncm.lin_reg_r_metric(window.W_A,window.W_B,
#             'cos': result_2
#         }
        print(results)
        #forward to OSC bundle
        send_osc_bundle(0, results)

#     for line in dataframe.values:
#         self.txtFile.addData('\n')
#         self.txtFile.addData(",".join([str(x) for x in line]))

In [5]:
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/*/bitalino", bitalino_input_handler)

In [6]:
print("Serving on {}".format(server.server_address))
server.serve_forever()

Serving on ('127.0.0.1', 8001)


KeyboardInterrupt: 